# GWHalophyte Sensitivity Analysis Notebook

This notebook runs a sensitivity analysis on the halophyte groundwater model using weather data from Agadir.

**Key Parameters:**
- Duration: Simulation length in days
- Initial soil moisture, storage volume, groundwater depth, salt concentrations
- Multiple parameter sets to explore sensitivity

**Output:**
- CSV file with model results (soil moisture, potentials, fluxes)
- Plots: soil moisture, fluxes, water potentials

In [8]:
# Import required libraries
from genericpath import exists
import os
from scipy.optimize import * 
from sympy import *
import numpy as np
import pandas as pd
from math import exp, pi, sqrt, log
import matplotlib.pyplot as plt
from dics import *
from functions import *
import importlib
import defs
importlib.reload(defs)
from defs import *
from previous_code.GWHalophyteFixingStorageEqn import *
import openpyxl
from datetime import date

print("Libraries imported successfully")

Libraries imported successfully


## Configuration & Parameter Setup

Define simulation parameters and sensitivity analysis ranges.

In [9]:
today = date.today()
print("Today's date:", today)

# Simulation parameters
duration = 20  # days
weatherFile = r"sample_data\AgadirInterp30.xlsx"
resultsFolder = r'sample_output\Diagnostic_{}_GWHalophyteSSensitivityAnalysis\\'.format(today)
resultsFile = resultsFolder + 'AgadirGWHalophyte_Diagnostic_020726.csv'

# Create results folder if it doesn't exist
if not os.path.exists(resultsFolder):
    os.makedirs(resultsFolder)

# Timestep configuration
timestepM = 30  # Model timestep (minutes)
timestepD = 30  # Input data timestep (minutes)
dt = timestepM * 60.0  # Convert to seconds for differential equations

# Sensitivity analysis parameter ranges
gw_s = [150,]              # Initial groundwater salt concentration (mol/m³)
s_s = [150,]               # Initial soil salt concentration (mol/m³)
plant_s = [150,]           # Initial plant salt concentration (mol/m³)
lam_s_x = [4000,]          # Soil root length density (m/m³)
lam_gw_x = [4000,]         # Tap root length density (m/m³)

print(f"Simulation duration: {duration} days")
print(f"Results will be saved to: {resultsFile}")
print(f"Number of parameter sets: {len(gw_s)}")

Today's date: 2026-02-08
Simulation duration: 20 days
Results will be saved to: sample_output\Diagnostic_2026-02-08_GWHalophyteSSensitivityAnalysis\\AgadirGWHalophyte_Diagnostic_020726.csv
Number of parameter sets: 1


## Load Weather Data

Read weather input from Excel file (temperature, humidity, solar radiation).

In [10]:
# Load weather data from Excel
df = pd.read_excel(weatherFile, engine='openpyxl')

# Extract and convert temperature to Kelvin
tempC = df['Temperature']  # in Celsius
taInp = tempC + 273.0
taInp = list(taInp.values)

# Extract relative humidity and convert to specific humidity (kg/kg)
rh = df['Relative Humidity']  # in %
psat = A_SAT * np.exp((B_SAT * (tempC)) / (C_SAT + tempC))  # Saturated vapor pressure (Pa)
qaInp = 0.622 * rh / 100.0 * psat / P_ATM
qaInp = list(qaInp.values)

# Extract solar radiation (global horizontal irradiance)
phiInp = list(df['GHI'].values)  # W/m²

print(f"Weather data loaded: {len(taInp)} timesteps")
print(f"Temperature range: {min(tempC):.1f} - {max(tempC):.1f} °C")
print(f"Solar radiation range: {min(phiInp):.1f} - {max(phiInp):.1f} W/m²")

Weather data loaded: 17521 timesteps
Temperature range: 11.0 - 33.1 °C
Solar radiation range: 0.0 - 781.9 W/m²


## Initialize Model

Set up species, soil, atmosphere, and hydraulic components.

In [ ]:
# Loop over sensitivity analysis parameter sets (typically just one for testing)
for x in range(0, 1):
    # Initial conditions
    sinit = 0.5                  # Initial soil moisture (-)
    vwi = 0.95                   # Initial storage volume (fraction of VWT)
    gw_z_init = 3.0              # Initial depth to groundwater table (m)
    cs_init = s_s[x]             # Initial soil salt concentration (mol/m³)
    cgw_init = gw_s[x]           # Initial groundwater salt concentration (mol/m³)
    cw_init = plant_s[x]         # Initial plant storage salt concentration (mol/m³)

    # Initialize species, atmosphere, soil, and photosynthesis
    species = Pmenz()            # Species: Pseudotsuga menziesii (Douglas fir)
    atmosphere = Atmosphere(phiInp[0], taInp[0], qaInp[0])
    soil = SaltySoilGW(Loam(), DrydownSoil(), species.ZR, sinit, cs_init, cgw_init, gw_z_init)
    photo = C3(species, atmosphere)
    
    # Initialize hydraulics (halophyte model with groundwater)
    hydro = HalophyteGW(species, atmosphere, soil, photo, vwi, cw_init)
    
    # Create simulation object
    plant = Simulation(species, atmosphere, soil, photo, hydro)
    
    print(f"Model initialized:")
    print(f"  Species: {species.NAME}")
    print(f"  Soil: Loam")
    print(f"  Initial soil moisture: {sinit}")
    print(f"  Initial plant storage concentration: {cw_init} mol/m³")

## Run Simulation

Advance the model through each timestep using weather data.

In [ ]:
    # Run simulation for each timestep
    print(f"\nRunning simulation for {duration} days...")
    n_steps = steps(duration, int(timestepM))
    
    for i in range(n_steps):
        plant.update(dt, phiInp[i], taInp[i], qaInp[i])
        
        # Progress indicator
        if (i + 1) % (n_steps // 4) == 0 or i == 0:
            print(f"  Completed {i+1}/{n_steps} timesteps ({(i+1)/n_steps*100:.1f}%)")
    
    print("Simulation complete!")
    
    # Extract results
    results = plant.output()
    
    # Save to CSV
    data = pd.DataFrame.from_dict(results)
    data.to_csv(resultsFile)
    print(f"\nResults saved to: {resultsFile}")

## Generate Plots

Create visualizations of soil moisture, fluxes, and water potentials.